In [13]:
import torch
import torchvision
import os
import torchvision.transforms as transforms
import numpy as np
import torch.nn as nn
import torch.optim as optim
from PIL import Image
from torch.utils.data.sampler import SubsetRandomSampler
from torch.utils.data import Dataset, DataLoader

In [14]:
class DiabeticRetinopathyDataset(Dataset):
    def __init__(self, base_dir, transform=None):
        self.base_dir = base_dir
        self.transform = transform
        self.data = []
        categories = ['No_DR', 'Mild', 'Moderate', 'Severe', 'Proliferate_DR']

        for idx, category in enumerate(categories):
            path = os.path.join(base_dir, category)
            images = os.listdir(path)[:1000]  # Take only 1000 images per category
            for image in images:
                self.data.append((os.path.join(path, image), idx))

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        img_path, label = self.data[idx]
        image = Image.open(img_path).convert('RGB')
        if self.transform:
            image = self.transform(image)
        return image, label


In [15]:
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

# Assuming 'base_dir' is your dataset directory path
base_dir = 'data/gaussian_filtered_images/gaussian_filtered_images/'  # Update this path
dataset = DiabeticRetinopathyDataset(base_dir=base_dir, transform=transform)

# Splitting the dataset into train and validation
train_size = int(0.8 * len(dataset))
val_size = len(dataset) - train_size
train_dataset, val_dataset = torch.utils.data.random_split(dataset, [train_size, val_size])

# Create data loaders
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)


In [18]:
import torch
import torch.nn as nn
import torchvision.models as models

# Load the pre-trained VGG16 model
model = models.vgg16(pretrained=True)

# Freeze the parameters of the feature extractor
for param in model.features.parameters():
    param.requires_grad = False

# Modify the classifier to fit your dataset (assuming 5 classes here)
num_features = model.classifier[6].in_features  # Get the input features of the last layer
features = list(model.classifier.children())[:-1]  # Remove the last layer
features.extend([nn.Linear(num_features, 5)])  # Append a new layer with 5 outputs
model.classifier = nn.Sequential(*features)  # Replace the classifier with the updated one

# Move the model to the appropriate device (GPU or CPU)
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model = model.to(device)

# Define the loss function
criterion = nn.CrossEntropyLoss()

# Define the optimizer (only update the parameters of the classifier)
optimizer = torch.optim.Adam(model.classifier.parameters(), lr=0.001)


Downloading: "https://download.pytorch.org/models/vgg16-397923af.pth" to /home/pavan/.cache/torch/hub/checkpoints/vgg16-397923af.pth


  0%|          | 0.00/528M [00:00<?, ?B/s]

In [19]:
epochs = 10

for epoch in range(epochs):
    model.train()
    running_loss = 0.0
    
    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
    
    model.eval()
    val_loss = 0.0
    correct = 0
    total = 0
    
    with torch.no_grad():
        for inputs, labels in val_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            val_loss += loss.item()
            
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    
    print(f'Epoch {epoch+1}, Train Loss: {running_loss / len(train_loader)}, Validation Loss: {val_loss / len(val_loader)}, Validation Accuracy: {100 * correct / total}%')


Epoch 1, Train Loss: 1.3091132822963927, Validation Loss: 1.3728201852904425, Validation Accuracy: 65.73426573426573%
Epoch 2, Train Loss: 1.0425565822256937, Validation Loss: 1.066296613878674, Validation Accuracy: 64.16083916083916%
Epoch 3, Train Loss: 1.0500584178500705, Validation Loss: 0.888153549697664, Validation Accuracy: 68.35664335664336%
Epoch 4, Train Loss: 1.005998203323947, Validation Loss: 0.9868426654073927, Validation Accuracy: 66.43356643356644%
Epoch 5, Train Loss: 1.046581512524022, Validation Loss: 0.9414576888084412, Validation Accuracy: 68.88111888111888%
Epoch 6, Train Loss: 0.9826627195709281, Validation Loss: 0.8358578003115125, Validation Accuracy: 69.93006993006993%
Epoch 7, Train Loss: 0.9021041898263825, Validation Loss: 1.0065177513493433, Validation Accuracy: 69.58041958041959%
Epoch 8, Train Loss: 0.8250648917423354, Validation Loss: 1.0562109053134918, Validation Accuracy: 69.4055944055944%
Epoch 9, Train Loss: 0.8899451171358427, Validation Loss: 1.1